# Fundamentals of Computer Science

## Lecture 11: Machine Learning

We start off very simply by loading a dataset from scikit-learn. In this case we are loading the [20 newsgroups dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html?highlight=newsgroups#sklearn.datasets.fetch_20newsgroups).


In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=["comp.graphics"], shuffle=True, random_state=42)
print(twenty_train.data[0])

From: bbs.mirage@tsoft.net (Jerry Lee)
Subject: Cobra 2.0 1-b-1 Video card HELP ME!!!!
Organization: The TSoft BBS and Public Access Unix, +1 415 969 8238
Lines: 22

Does ANYONE out there in Net-land have any information on the Cobra 2.20 
card?  The sticker on the end of the card reads
        Model: Cobra 1-B-1
        Bios:  Cobra v2.20

I Havn't been able to find anything about it from anyone!  If you have 
any information on how to get a hold of the company which produces the 
card or know where any drivers are for it, PLEASE let me know!

As far as I can tell, it's a CGA card that is taking up 2 of my 16-bit 
ISA slots but when I enable the test patterns, it displays much more than 
the usualy 4 CGA colors... At least 16 from what I can count.. Thanks!

              .------------------------------------------.
              : Internet: jele@eis.calstate.edu          :
              :           bbs.mirage@gilligan.tsoft.net  :
              :           bbs.mirage@tsoft.sf-bay.org

There is a "problem": The class labels are actually `int`s. We can "restore" the original string labels to make them human-readable again.

In [ ]:
import pandas as pd
train_df = pd.DataFrame(twenty_train.data, columns=["text"])
# targets = [twenty_train.target_names[x] for x in twenty_train.target]
targets = twenty_train.target
train_df['target'] = pd.Series(data=targets)

In [ ]:
train_df

We need **features** to train our classifier with. One simple way to extract features from text data is to create a **bag-of-words** model. It simply counts how often each word appears in a text.

This means we ignore the grammar completely! In practice however this approach works reasonably well for text classification though better and more sophisticated approaches exist. It is nevertheless a good starting point when you first start doing machine learning tasks. 

In [ ]:
from collections import Counter
lst = ["Hello", "World", "Goodbye", "World"]
c = Counter(lst)
print(c)
print(c["Does not exist"])

Here we use the `CountVectorizer` to do this process for us. It takes all the texts, "learns" which words exist and creates a *sparse* matrix where each row is a document and each column is a word, and the values in the cells are the word frequency of that word for that particular text.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df["text"])
print(X_train.shape)
print(len(train_df))

In [ ]:
X_train

tf-idf stands for *term frequency — inverse document frequency*

Now let's look at what is happening a bit more in-depth. Here, we have 4 documents.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

data_set = (
    "Bali is an island and not a country",
    "Peru is a country in south america",
    "This is a country. That is not a country",
    "japan is a country made of more than one island"
)

vectorizer = CountVectorizer(stop_words='english')

vectorizer.fit(data_set)

print(vectorizer.vocabulary_)

In [ ]:
# Converting dataset into spare matrix
smatrix = vectorizer.transform(data_set)

print(smatrix)
print(smatrix.shape)
# Note that the sparse matrix created called smatrix is a scipy sparse matrix with
# elements stored in coordinate format. But you can convert it into a dense format.
# Dense representation of the above sparse matrix

print(vectorizer.vocabulary_)

smatrix.todense()

Each row in the output matrix corresponds to document in the data_set. Each column corresponds to corresponding words from the vocabulary. 
For example: column 0 is for ‘america’. column 1 is for ‘bali’ and so on.

Each element of the matrix is ‘term frequency’, i.e. number of times the word appears in that particular document.
For Example: ‘country’ appears 2 times in document 3.

Finally, idf (inverse document frequency)


$$ idf_t = \log \frac{1+N}{1+df_t}  $$

$N$ cardinality of our document space. In this example it is `4`.

$df_t$ the number of documents that term t appears in **(adding 1 into the formula to avoid zero-division)**.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer() #by default norm = "l2"

tfidf.fit(smatrix)

print("IDF:", tfidf.idf_)

IDF is calculated for each feature (in this case each term t)

idf for term 'island':

\begin{align} 
N&=4\\
df_t  =  df(\text{island}) & = 2 \qquad \text{Island appears in document 1 & 4}\\
idf & =  \log \frac{1+4}{1+2}+1\\
& = log \frac{5}{3}+1\\
& = 0.51 + 1\\
& = 1.51 
\end{align}

Using the above IDF, tf-idf is calculated by multiplying it with term frequency matrix.

tf-idf calculation of document 1

\begin{align}
\vartheta_{tfidf} & = 
\begin{bmatrix}
 0 & 1 & 1 & 1 & 0 & 0 & 0
\end{bmatrix}
\times
\begin{bmatrix}
1.916 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1.916 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1.916 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1.916 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1.916 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1.916 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1.916
\end{bmatrix} \\
\vartheta_{tfidf} & = \begin{bmatrix}
 0 & 1.916 & 1 & 1.51 & 0 & 0 & 0
\end{bmatrix}
\end{align}

Final step is to normalize each row of this matrix. This is to overcome a bias towards frequently repeated words in long documents which might make them look more important than they are just because of the high frequency of the term in the document.

Please note: `TfidfTransformer` does the above calculation and L2 normalization (by default) on the `smatrix`.

L2 normalization is the most famous method to normalize. Below is an image where L2 norm calculation is worked out on row1 for the above matrix.

\begin{align}
\vec{tfidf_{doc1}} & = \begin{pmatrix} 
0 & 1.916 & 1 & 1.51 & 0 & 0 & 0 
\end{pmatrix}
\end{align}

\begin{align}
\vartheta_{tfidf}  & = \begin{bmatrix}
 0 & 1.916 & 1 & 1.51 & 0 & 0 & 0
\end{bmatrix} \quad \text{without norm} \\
\text{Apply L2 norm} \\
\vartheta_{norm}  & = \frac{ \begin{bmatrix}
 0 & 1.916 & 1 & 1.51 & 0 & 0 & 0
\end{bmatrix}}{\sqrt{1.916^2+1^2+1.52^2}}
= 
\frac{ \begin{bmatrix}
 0 & 1.916 & 1 & 1.51 & 0 & 0 & 0
\end{bmatrix}}{2.63} \\
\vartheta_{norm}  & = 
\begin{bmatrix}
 0 & 0.726 & 0.379 & 0.572 & 0 & 0 & 0
\end{bmatrix} \quad \text{for document 1} 
\end{align}

Final matrix after normalization is

In [ ]:
tf_idf_matrix = tfidf.transform(smatrix)

print(tf_idf_matrix.todense())

- Larger the value, more important is the term for that document.
- If a particular term say ‘t1’ exists in all the documents, along with some unique terms in a particular document. ‘t1’ is assigned less weightage than the unique terms in that document.
For example: in the first document 1 = ‘Bali is an island and not a country’ 
for tf_idf_matrix we can see row 1 has term ‘country’ assigned less weightage (=0.379) when compared to term ‘bali’ with weightage of 0.726 which exists only in that document. term ‘island’ is given weightage of 0.572 which is between that of ‘bali’ and ‘country’ as it appears in document 4 as well.
- tf-idf matrix can be used as a feature set for your logistic regression or decision tree based machine learning algorithms.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

print(tfidf.fit_transform(data_set).todense())

One important step we technically don't **have to** do, but **absolutely should do**, is split our data into a training and a test set. To train our machine learning model we need to give it some data. 

But then of course we want to know how good our model is. For classification we can give our model some data where we know the class that the model should give us, and compare what the model tells us (the prediction) with the correct solution (that we know, because we ask the model to give us a prediction for data we already know the answer to).

We could train our model on the data, and then ask the model to predict the class for the data that it was trained on, but then the model could just "cheat" and remember the solution from when it saw this data during training.

Instead what we do is to keep some of our data for **testing** and use some of the data for **training**. We train our model on the training data, and then we use the testing data to evaluate the model.

We ask the model to predict the output for our testing data. During this process it doesn't see the correct output! The testing data is data the model has never seen before. But we know the correct output, so we can compare the output of the model with the correct output and for example calculate in % how often the model told us the correct output.

Then, if we train the model again with different parameters, or if we train a completely different model on the same data, we can compare which of the models is the best one, based on how often they predicted correctly.

scikit-learn has a very convenient method that helps us split the data into a training and testing set, but of course we could do this by hand too. It's just more convenient to use this method.

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
train_df = pd.DataFrame(twenty_train.data, columns=["text"])
targets = [twenty_train.target_names[x] for x in twenty_train.target]
train_df['target'] = pd.Series(data=targets)
t_train, t_test = train_test_split(train_df,train_size=0.6)
print(len(t_train), len(t_test), type(t_train))

In [ ]:
t_train

Next we will take a look at the [IRIS dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html).

In [ ]:
from sklearn import datasets
import pandas as pd

iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df.head()

In [ ]:
labels = pd.DataFrame(iris.target)
labels.sample(5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :2], labels[0])
X_train.shape

Here in this cell is the entire code we need to train a classifier!

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
n_neighbors = 3
clf = KNeighborsClassifier(n_neighbors)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
predictions[:10]

Of course we want to know how good our model is. Here we calculate the **F1** metric, and we also perform **cross-validation**.

More specifically, we perform **k-fold cross-validation** here. We split our data into 10 blocks, and each block is used as a testing set once, all the other blocks are used for training. That means we train and evaluate our model 10 times!

Then, for each training run, we get the **F1** score, here macro-averaged. The macro here means that we calculate the F1-score for *each class independently*, and then average. So in total we get 10 F1-scores, one for each training run. This gives us a good indication whether or not our model is actually good. We could have a situation where the data we use for testing is very easy, and so our model gets a very high score, but it fails for every other case. We could notice that if we see that one of the 10 scores is much higher than the rest, and then if we average the 10 scores we have a much better understanding of the quality of our model.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import cross_val_score

X = fetch_20newsgroups(shuffle=True)

vec = TfidfVectorizer()
X_data = vec.fit_transform(X["data"])
clf = PassiveAggressiveClassifier()
cross_val_score(clf, X_data, X["target"], cv=10, scoring="f1_macro")

We have talked about the F1-score, but not how to calculate it. To calculate it we actually need two other metrics, the **Precision** and the **Recall**.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

train = fetch_20newsgroups(subset='train', shuffle=True)
test = fetch_20newsgroups(subset='test', shuffle=True)

vec = TfidfVectorizer()
X_train = vec.fit_transform(train["data"])
clf = PassiveAggressiveClassifier()
clf.fit(X_train, train["target"])

X_test = vec.transform(test["data"])
y_test = test["target"]
predictions = clf.predict(X_test)

First though we must understand a few key concepts: True Positives (TP), False Positives (FP), False Negatives (FN) and True Negatives (TN).


*   TP: An element belongs to a class, and the classifier predicted it as this class.
*   FP: An element does **not** belong to a class, but the classifier predicted it as this class.
*   FN: An element belongs to a class, but the classifier did **not** predict it as this class.
*   TN: An element does **not** belong to a class, and the classifier did **not** predict it as this class.

These values are calculated from the "perspective" of a class.

Let's assume we have to classes, *Fruit* and *Vegetable*.

Fruit = Apple, Banana, Pineapple

Vegetable: Cucumber, Carrot, Onion

We have given our classifier these elements, and it is telling us the following:

Fruit* = Apple, Banana, Carrot, Onion

Vegetable* = Pineapple, Cucumber


How many TPs for Fruit do we have? For this we look at the predictions of our classifier. We count all those elements that are actually Fruits as TPs for the Fruit class: Apple, Banana. So we have 2 TPs for the Fruit class.

How many TPs for Vegetable do we have? Only one, Cucumber. Cucumber is a vegetable, and the classifier told us that it is a vegetable.

Next, the FPs for Fruit: When did our classifier say something is a Fruit, even though it is not? That happened for Carrot and Onion. So we have 2 FPs for the Fruit class.

What about Vegetable? Only one, Pineapple.

Next the False Negatives. For Fruit we must look at all elements that **are** Fruits, but that our classifier did not assign the Fruit class to: This happened only for Pineapple, so we have 1 FN for the Fruit class.

For Vegetable we have 2! Carrot and Onion are Vegetables, but the classifier did not predict the Vegetable class.

The TNs work exactly the same.


Let's begin with Precision. It tells us how often, when we predicted a class, was the classifier correct.

\begin{align}
P = \frac{TP}{TP+FP}
\end{align}

For Fruit, we get a Precision of: $$P_{Fruit} = \frac{2}{2 + 2} = 0.5$$

And for Vegetable: $$P_{Vegetable} = \frac{1}{1 + 1} = 0.5$$

This means that when our classifier predicted the Vegetable (or Fruit) class, it was right half the time, or 50%.

Next is the Recall. It tells us how often we have assigned the class out of how often we *should have* assigned the class.

$$R = \frac{TP}{TP + FN}$$

For Fruit, we get a Recall of: $$R_{Fruit} = \frac{2}{2 + 1} = 0.67$$ 

And for Vegetable: $$R_{Vegetable} = \frac{1}{1 + 2} = 0.33$$

This means that we assigned Fruit class to 67% of the elements that should have, but only did the same for the Vegetable class for 33% of the elements.

The F1-measure then combines these two metrics, to give us a more "balanced" score. In reality in some cases we care more about Precision than Recall, but often both are important.

$$F1 = 2 * \frac{P*R}{P+R}$$

Then there is another metric, the Accuracy.

$$A = \frac{TP + TN}{TP + FP + FN + FP}$$

Of course we can speed up this process of manual calculations with Python!

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

It's also useful to see where the classifier is going wrong specifically.

Let's look at the IRIS example again, which has 3 classes. 

We can create a **Confusion Matrix** that tells us as what classes the flowers were misclassified as.

The way to read a confusion matrix is that the rows represent the true labels, and the columns the predicted labels.

```
['setosa' 'versicolor' 'virginica']
array([[15,  0,  0],
       [ 0,  8,  3],
       [ 0,  5,  7]])
```

Here, the 15 means that 15 elements belong to the "setosa" class (because the value is in the first row), and they were classified as "setosa" (because the value is in the first column).

Let's look at the 3 in the second row. What does that mean?

We know it's in the second row, and the second row represents all elements that belong to the "versicolor" class. The 3 is in the third column, the third column represents "virginica". This means that 3 elements, that belong to the "versicolor" class, were classified wrongly as "virginica".

Some useful tips: If you sum up a row it tells you how many elements belong to the class in total, and if you sum up the column values then it tells you how many elements were **predicted** as that class.

For "virginica" we know that 0 + 3 + 7 elements were predicted as it, and that in total 0 + 5 + 7 elements belong to the class.

In [ ]:
from sklearn.metrics import confusion_matrix
print(iris.target_names)

from sklearn import datasets
import pandas as pd

iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :2], labels[0], random_state=42)
X_train.shape

y_train = [iris.target_names[x] for x in y_train]
y_test = [iris.target_names[x] for x in y_test]

from sklearn.neighbors import KNeighborsClassifier
n_neighbors = 3
clf = KNeighborsClassifier(n_neighbors)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
predictions[:10]


confusion_matrix(y_test, predictions, labels=iris.target_names)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average="macro")
recall = recall_score(y_test, predictions, average="macro")
f1 = f1_score(y_test, predictions, average="macro")
print(accuracy, precision, recall, f1)

To find the ideal parameters of a classifier we can perform a gridsearch. You specify the parameters, and possible values, and then scikit-learn tries out the combinations for you and tells you which combination has performed the best.

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
    "C": [x/10 for x in range(0, 3)],
    "fit_intercept": [True, False],
    "max_iter": [x*1000 for x in range(1, 2)],
    "shuffle": [False],
    "random_state": [42],
    "tol": [0.0001, 0.001, 0.01, 0.1],
}
clf = PassiveAggressiveClassifier(random_state=42, shuffle=False)
clf = GridSearchCV(clf, parameters, cv=5, scoring="precision")

# This can take a while!
# clf.fit(X_train, train["target"])
# print(clf.best_params_)
# {'C': 0.2, 'fit_intercept': True, 'max_iter': 1000, 'random_state': 42, 'shuffle': False, 'tol': 0.001}